Código obtenido de modeling_atom.ipynb y electronic_energy.ipynb

In [1]:
from qiskit_algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit_nature.units import DistanceUnit
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms.utils import algorithm_globals
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_nature.second_q.mappers import JordanWignerMapper

In [2]:
driver = PySCFDriver(
    atom="H 0 0 0",
    basis="sto3g",
    charge=0,
    spin=1,
    unit=DistanceUnit.ANGSTROM,
)

problem = driver.run()
hamiltonian = problem.hamiltonian

/home/tfg/TFG/tfg_env/lib/python3.10/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [3]:
hamiltonian = problem.hamiltonian

coefficients = hamiltonian.electronic_integrals
print(coefficients.alpha)

Polynomial Tensor
 "+-":
array([[-0.46658185]])
 "++--":
array([0.77460594])


In [22]:
mapper = JordanWignerMapper()
fermionic_op = hamiltonian.second_q_op()
qubit_op = mapper.map(fermionic_op)

# define callback
# note: Re-run this cell to restart lists before training
counts = []
values = []

def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

seed = 170
algorithm_globals.random_seed = seed
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)

iterations = 125
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
spsa = SPSA(maxiter=iterations)

vqe = VQE(
    noiseless_estimator, ansatz, optimizer=spsa, callback=store_intermediate_result
)

result_vqe = vqe.compute_minimum_eigenvalue(operator=qubit_op)

In [23]:
result_vqe.eigenvalue.real + hamiltonian.nuclear_repulsion_energy

-0.4665818495572753